In [ ]:
%pip install ultralytics
%pip install albumentations

In [ ]:
import os
import shutil
import torch
from google.colab import drive
from ultralytics import YOLO
import albumentations as A
import cv2


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
drive.mount('/content/gdrive/')

!unzip -q /content/gdrive/MyDrive/data.zip


Mounted at /content/gdrive/


In [ ]:
# data.yaml

data_yaml = """
train: ./train
val: ./test
nc: 11
names: [
    '1_отвертка_минус',           # Отвертка «-»
    '2_отвертка_плюс',            # Отвертка «+»
    '3_отвертка_смещенный_крест', # Отвертка на смещенный крест
    '4_коловорот',                # Коловорот
    '5_пассатижи_контровочные',   # Пассатижи контровочные
    '6_пассатижи',                # Пассатижи
    '7_шэрница',                  # Шэрница
    '8_разводной_ключ',           # Разводной ключ
    '9_открывашка_для_банок',     # Открывашка для банок с маслом
    '10_ключ_рожковый_накидной',  # Ключ рожковый накидной ¾
    '11_бокорезы',                # Бокорезы
  ]
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

In [ ]:
def count_images(folder_path):
    """Подсчитывает количество jpg файлов в указанной папке"""
    jpg_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.jpg')]
    return len(jpg_files)

train_count = count_images('./train')
test_count = count_images('./test')

print(f"Количество фото в папке train: {train_count}")
print(f"Количество фото в папке test: {test_count}")
print(f"Общее количество фото: {train_count + test_count}")


Количество фото в папке train: 149
Количество фото в папке test: 36
Общее количество фото: 185


In [ ]:
def get_augmentations():
    return A.Compose([
        # ГЕОМЕТРИЧЕСКИЕ ПРЕОБРАЗОВАНИЯ
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
        A.RandomRotate90(p=0.4),
        A.Rotate(limit=30, p=0.6),
        A.ShiftScaleRotate(
            shift_limit=0.2,
            scale_limit=0.3,
            rotate_limit=25,
            p=0.7
        ),

        # ИСКАЖЕНИЯ И ИЗЛОМЫ
        A.Perspective(
            scale=(0.03, 0.10),  # перспективные искажения
            p=0.5
        ),
        A.ElasticTransform(
            alpha=120,           # сильные эластичные деформации
            sigma=120 * 0.05,
            alpha_affine=120 * 0.03,
            p=0.3
        ),
        A.GridDistortion(
            num_steps=10,       # сеточные искажения
            distort_limit=0.3,  # усиленная дисторсия
            p=0.4
        ),
        A.OpticalDistortion(
            distort_limit=0.3,  # оптические искажения
            shift_limit=0.2,
            p=0.3
        ),

        # ОСВЕЩЕНИЕ И ЦВЕТ
        A.RandomBrightnessContrast(
            brightness_limit=0.4,  # увеличен диапазон яркости
            contrast_limit=0.4,    # увеличен контраст
            brightness_by_max=True,
            p=0.8
        ),
        A.HueSaturationValue(
            hue_shift_limit=30,    # увеличен сдвиг оттенка
            sat_shift_limit=40,    # увеличенная насыщенность
            val_shift_limit=40,    # увеличенная яркость
            p=0.8
        ),
        A.CLAHE(clip_limit=4.0, p=0.3),  # контрастная ограниченная адаптивная гистограмма
        A.RandomGamma(gamma_limit=(60, 140), p=0.4),  # гамма-коррекция

        # РАЗМЫТИЯ
        A.GaussianBlur(blur_limit=(3, 9), p=0.5),     # увеличен лимит размытия
        A.MotionBlur(blur_limit=(5, 15), p=0.3),      # размытие в движении
        A.MedianBlur(blur_limit=7, p=0.2),            # медианное размытие
        A.GlassBlur(sigma=0.7, max_delta=4, iterations=2, p=0.2),  # стеклянное размытие

        # ШУМЫ И ПОМЕХИ
        A.GaussNoise(
            var_limit=(20.0, 80.0),    # увеличенный шум
            p=0.4
        ),
        A.ISONoise(
            color_shift=(0.01, 0.05),  # шум ISO
            intensity=(0.1, 0.5),
            p=0.3
        ),
        A.MultiplicativeNoise(
            multiplier=(0.9, 1.1),     # мультипликативный шум
            per_channel=True,
            p=0.2
        ),

        # УХУДШЕНИЕ КАЧЕСТВА
        A.ImageCompression(
            quality_lower=30,      # сильное сжатие JPEG
            quality_upper=70,
            p=0.4
        ),
        A.Downscale(
            scale_min=0.3,         # уменьшение разрешения
            scale_max=0.7,
            p=0.3
        ),

        # АТМОСФЕРНЫЕ ЭФФЕКТЫ
        A.RandomFog(
            fog_coef_lower=0.1,    # эффект тумана
            fog_coef_upper=0.4,
            alpha_coef=0.08,
            p=0.2
        ),
        A.RandomShadow(
            shadow_roi=(0, 0.5, 1, 1),  # случайные тени
            num_shadows_lower=1,
            num_shadows_upper=3,
            shadow_dimension=8,
            p=0.3
        ),

        # ВЫРЕЗАНИЯ И ЗАКРЫТИЯ
        A.CoarseDropout(
            max_holes=10,          # увеличенное количество отверстий
            max_height=20,         # увеличенный размер
            max_width=20,
            min_holes=2,
            min_height=4,
            min_width=4,
            fill_value=0,
            p=0.4
        )

    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


In [ ]:
# Создание аугментированных данных
def create_augmented_data():
    aug = get_augmentations()
    augmented_count = 0

    for img_name in os.listdir('./train'):
        if not img_name.lower().endswith(('.jpg', '.jpeg')):
            continue

        # Пути для каждого фото
        img_path = os.path.join('./train', img_name)
        label_path = os.path.join('./train', os.path.splitext(img_name)[0] + '.txt')

        # Загрузка изображения
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        bboxes, class_labels = [], []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    if line.strip():
                        cls, x, y, w, h = map(float, line.split())
                        bboxes.append([x, y, w, h])
                        class_labels.append(int(cls))

        # Применение аугментации
        if bboxes:
            augmented = aug(image=image, bboxes=bboxes, class_labels=class_labels)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_class_labels = class_labels  # Сохраняем оригинальные метки классов
        else:
            aug_image = image
            aug_bboxes = bboxes
            aug_class_labels = class_labels

        # Сохранение аугментированных данных
        base_name = os.path.splitext(img_name)[0]
        aug_img_path = os.path.join('./train', f'{base_name}_aug.jpg')
        aug_label_path = os.path.join('./train', f'{base_name}_aug.txt')

        cv2.imwrite(aug_img_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))

        with open(aug_label_path, 'w', encoding='utf-8') as f:
            for bbox, cls_id in zip(aug_bboxes, aug_class_labels):
                f.write(f"{cls_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

        augmented_count += 1

    return augmented_count

print("Создание аугментированных данных...")
augmented_count = create_augmented_data()
print(f"Успешно создано аугментированных фото: {augmented_count}")

Создание аугментированных данных...


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-736876611.py:20: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(
/tmp/ipython-input-736876611.py:31: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(
/tmp/ipython-input-736876611.py:60: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(
/tmp/ipython-input-736876611.py:76: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(
/tmp/ipython-input-736876611.py:81: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(
/tmp/ipython-input-736876611.py:88: UserWarning: Argument(s) '

Успешно создано аугментированных фото: 149


In [ ]:
model = YOLO('yolov8m.pt')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch = 8 if device == 'cuda' else 4


In [ ]:
# Обучение
results = model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=batch,
    device=device,
    workers=0
)


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, 

In [ ]:
# Обученная модель
best_model = YOLO('runs/detect/train/weights/best.pt')

# Конвертация в ONNX
best_model.export(
    format="onnx",
    opset=12,
    simplify=True,
    imgsz=640,
    dynamic=False
)


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 8.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1rc1 opset 12...
ONNX: slimming with onnxslim 0.1.70...
ONNX: export success ✅ 13.2s, saved as 'runs/detect/train/weights/best.onnx' (98.8 MB)

Export complete (17.1s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict t

'runs/detect/train/weights/best.onnx'

In [ ]:
# Сохраняем результаты в Google Drive
save_path = '/content/gdrive/MyDrive/yolo_results'
os.makedirs(save_path, exist_ok=True)

files_to_copy = [
    'runs/detect/train/weights/best.pt',       # лучшая модель
    'runs/detect/train/weights/best.onnx',     # лучшая модель
    'runs/detect/train/results.png',           # графики обучения
    'runs/detect/train/confusion_matrix.png',  # матрица ошибок
    'runs/detect/train/args.yaml'              # аргументы обучения
]

for file_path in files_to_copy:
    if os.path.exists(file_path):
        shutil.copy(file_path, save_path)

print(f"Результаты сохранены в: {save_path}")

Результаты сохранены в: /content/gdrive/MyDrive/yolo_results


In [ ]:
# Оценка
metrics = best_model.val()

# Сводка метрик
print("ОСНОВНЫЕ МЕТРИКИ:")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

f1 = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr) if (metrics.box.mp + metrics.box.mr) > 0 else 0
print(f"F1-score: {f1:.4f}")

# Метрики по классам в табличном формате
print("\nМЕТРИКИ ПО КЛАССАМ:")
print("Класс".ljust(30) + "Precision".ljust(20) + "Recall".ljust(10) + "F1-score")
print("-" * 50)

for i, class_name in enumerate(metrics.names.values()):
    p = metrics.box.p[i]
    r = metrics.box.r[i]
    f1_class = 2 * (p * r) / (p + r) if (p + r) > 0 else 0
    print(f"{class_name.ljust(30)}{p:.4f}".ljust(40) + f"{r:.4f}".ljust(10) + f"{f1_class:.4f}")

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5050.4±791.1 MB/s, size: 6865.3 KB)
val: Scanning /content/test.cache... 36 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 36/36 61.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 8.0s
                   all         36        395      0.871      0.793       0.87      0.696
      1_отвертка_минус         36         36      0.894      0.468      0.662      0.515
       2_отвертка_плюс         36         36      0.679      0.583      0.637       0.52
3_отвертка_смещенный_крест         36         36      0.822      0.833       0.89       0.78
           4_коловорот         36         36      0.904      0.944      0.982      0.838
5_пассатижи_контровочные         36         36      0.945     